In [20]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define the path to your image dataset
image_folder = "C:/Users/saith/OneDrive/Desktop/DWDM/Images"

# Mapping of emotions to numerical labels
emotion_map = {
     'neutral':0,
     'calm':1,
     'happy':2,
     'sad':3,
     'angry':4,
     'fearful':5,
     'disgust':6,
     'surprised':7
}

# Function to extract emotion from the filename
def get_emotion_from_filename(filename):
    # Assuming filenames contain the emotion (adjust logic if needed)
    filename=os.path.basename(filename).split('-')[0]
    for emotion in emotion_map:
        if emotion in filename.lower():
            return emotion_map[emotion]
    raise ValueError("Emotion label not found in filename")

# Load images and labels
image_data = []
labels = []
for file in os.listdir(image_folder):
    if file.endswith('.jpg') or file.endswith('.png'):
        # Read image
        img_path = os.path.join(image_folder, file)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (128, 128))  # Resize to fixed size for CNN
        image_data.append(image)

        # Get emotion from the filename
        emotion = get_emotion_from_filename(file)
        labels.append(emotion)

# Convert to numpy arrays and normalize the images
image_data = np.array(image_data) / 255.0  # Normalize pixel values to [0, 1]
labels = np.array(labels)

# Convert labels to categorical (one-hot encoding for multi-class classification)
num_classes = len(emotion_map)
labels = to_categorical(labels, num_classes=num_classes)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)



(1152, 8)
(288, 8)


In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
from tensorflow.keras.layers import LSTM

# Build the CNN model
model = Sequential()

# Use Input layer explicitly to define the input shape
model.add(Input(shape=(128, 128, 3)))  # Input size (128x128 RGB image)

# Convolutional Layer 1
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the layers
model.add(Flatten())

# Reshape to add RNN layer
# Assuming the RNN layer expects an input of shape (timesteps, features),
# we'll reshape the flattened output to fit this requirement.
# Here we assume timesteps = 1, features = flattened size
model.add(Reshape((1, -1)))  # Reshape to (timesteps, features)

# Add RNN Layer
model.add(LSTM(64, return_sequences=False))  # You can use SimpleRNN or GRU as well

# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization

# Output Layer (for multi-class classification)
num_classes = 8  # Assuming 8 emotions (adjust based on your dataset)
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [39]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


Epoch 1/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 17s 478ms/step - accuracy: 0.9431 - loss: 0.1752 - val_accuracy: 0.5451 - val_loss: 1.6606
Epoch 2/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 17s 485ms/step - accuracy: 0.9590 - loss: 0.1405 - val_accuracy: 0.5625 - val_loss: 1.5889
Epoch 3/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 17s 464ms/step - accuracy: 0.9640 - loss: 0.1068 - val_accuracy: 0.5868 - val_loss: 1.6132
Epoch 4/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 16s 455ms/step - accuracy: 0.9872 - loss: 0.0692 - val_accuracy: 0.6007 - val_loss: 1.6555
Epoch 5/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 16s 454ms/step - accuracy: 0.9783 - loss: 0.0706 - val_accuracy: 0.5833 - val_loss: 1.6493
Epoch 6/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 17s 456ms/step - accuracy: 0.9650 - loss: 0.1143 - val_accuracy: 0.5729 - val_loss: 1.6468
Epoch 7/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 16s 446ms/step - accuracy: 0.9767 - loss: 0.0991 - val_accuracy: 0.5729 - val_loss: 1.7952
Epoch 8/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 16s 449ms/step - accuracy: 0.9732 - loss: 0.0823 - val_accu

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
